# Chapter8-2

## 教師ありSimCSEの実装

### ライブラリのインストール

In [1]:
!pip install datasets scipy transformers[ja,torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 71.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 

### 準備、データセットの読み込み、前処理

In [2]:
from transformers.trainer_utils import set_seed
set_seed(42)

In [3]:
# データセットの読み込み
from datasets import load_dataset
dataset = load_dataset("llm-book/jsnli", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for llm-book/jsnli contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/jsnli.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/533005 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3916 [00:00<?, ? examples/s]

In [4]:
# データを確認
print(dataset)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 533005
})


In [5]:
# データの詳細を確認
from pprint import pprint

pprint(dataset[0])
pprint(dataset[1])

{'hypothesis': '男 は 魔法 の ショー の ため に ナイフ を 投げる 行為 を 練習 して い ます 。',
 'label': 'neutral',
 'premise': 'ガレージ で 、 壁 に ナイフ を 投げる 男 。'}
{'hypothesis': '女性 が 畑 で 踊って い ます 。',
 'label': 'contradiction',
 'premise': '茶色 の ドレス を 着た 女性 が ベンチ に 座って い ます 。'}


In [6]:
import csv
import random
from typing import Iterator

# JSNLIの学習セットから、前提文、ラベルごとに仮説文をまとめたdictを作成
premise2hypothesis = {}

premises = dataset["premise"]
hypotheses = dataset["hypothesis"]
labels = dataset["label"]

for premise, hypothesis, label in zip(premises, hypotheses, labels):
    if premise not in premise2hypothesis:
        premise2hypothesis[premise] = {
         "entailment": [],
         "neutral": [],
         "contradiction": [],
        }
    premise2hypothesis[premise][label].append((hypothesis))

`premise2hypotheses` から前提文、含意ラベル、仮設ラベルの3つの組を生成するジェネレーター関数を定義

$\rightarrow$これによってデータセットを構築することができる

In [7]:
from datasets import Dataset

def generate_sup_train_example() -> Iterator[dict[str, str]]:
  """ 教師ありSimCSEの学習セットの事例を生成 """

  # データセットから3つの組を生成する
  for premise, hypotheses in premise2hypothesis.items():

    # 矛盾のラベルの仮説文が1つも存在しない事例はスキップ
    if len(hypotheses["contradiction"]) == 0:
      continue

    # 含意ラベルの仮説文1つにつき、矛盾ラベルの仮説文1つをランダムに関連付け
    for entialment_hypotheses in hypotheses["entailment"]:
      contradiction_hypotheses = random.choice(hypotheses["contradiction"])

      # 3つの組をdictとして生成
      yield{
          "premise": premise,
          "entailment_hypothesis": entialment_hypotheses,
          "contradiction_hypothesis": contradiction_hypotheses,
      }

# ジェネレータ関数を使い、教師ありSimCSEの学習セットを構築
sup_train_dataset = Dataset.from_generator(generate_sup_train_example)

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
#　作成した学習セットの形式と内容の確認
print(sup_train_dataset)

Dataset({
    features: ['premise', 'entailment_hypothesis', 'contradiction_hypothesis'],
    num_rows: 173438
})


In [9]:
# データセットの事例を表示
pprint(sup_train_dataset[0])
pprint(sup_train_dataset[1])

{'contradiction_hypothesis': '男 が 台所 の テーブル で 本 を 読んで い ます 。',
 'entailment_hypothesis': 'ガレージ に 男 が い ます 。',
 'premise': 'ガレージ で 、 壁 に ナイフ を 投げる 男 。'}
{'contradiction_hypothesis': '黒人 は デスクトップ コンピューター を 使用 し ます 。',
 'entailment_hypothesis': '人 は 椅子 に 座って い ます 。',
 'premise': 'ラップ トップ コンピューター を 使用 して 机 に 座って いる 若い 白人 男 。'}


### 検証用、テスト用のデータセットを読み込み

In [10]:
DATASET = "llm-book/JGLUE"

# 検証セットのダウンロード
valid_dataset = load_dataset(
    DATASET, name="JSTS", split="train"
)

# テストセットのダウンロード
test_dataset = load_dataset(
    DATASET, name="JSTS", split="validation"
)

The repository for llm-book/JGLUE contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/JGLUE.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

### collate関数の準備

In [11]:
from transformers import AutoTokenizer

base_model_name = "cl-tohoku/bert-base-japanese-v3"

# トークナイザを初期化
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

In [12]:
from transformers import BatchEncoding
from torch import Tensor
import torch
def sup_train_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
  """ 訓練セットのミニバッチを作成 """

  premises = []
  hypotheses = []

  for example in examples:
    premises.append(example["premise"])

    entailment_hypothesis = example["entailment_hypothesis"]
    contradiction_hypothesis = example["contradiction_hypothesis"]

    hypotheses.extend(
        [entailment_hypothesis, contradiction_hypothesis]
    )

    # ミニバッチに含まれる全体文と仮説文にトークナイザを適用する
    tokenized_premises = tokenizer(
        premises,
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )

    tokenized_hypotheses = tokenizer(
        hypotheses,
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )

    # 前提文・仮説文の類似度行列における政令ペアの位置を示すTensorを作成する
    # 行列のi行目の事例に対して 2 * i列目の要素が政令ペアとなる
    labels = torch.arange(0, 2 * len(premises), 2)

    return {
        "tokenized_texts_1": tokenized_premises,
        "tokenized_texts_2": tokenized_hypotheses,
        "labels": labels,
    }

In [13]:

def eval_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
  """ SimCSEの検証・テストセットのミニバッチを作成 """

  # ミニバッチの文ペアに含まれる文それぞれトークナイザを適用
  tokenized_texts_1 = tokenizer(
      [example["sentence1"] for example in examples],
      padding=True,
      truncation=True,
      max_length=512,
      return_tensors="pt"
  )

  tokenized_texts_2 = tokenizer(
      [example["sentence2"] for example in examples],
      padding=True,
      truncation=True,
      max_length=512,
      return_tensors="pt"
  )

  labels = torch.arange(len(examples))

  # 類似度スコアのTensorを作成
  label_scores = torch.tensor(
      [example["label"] for example in examples]
  )

  return {
      "tokenized_texts_1": tokenized_texts_1,
      "tokenized_texts_2": tokenized_texts_2,
      "labels": labels,
      "label_scores": label_scores
  }

### モデルの準備

In [23]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel
from transformers.utils import ModelOutput

class SimCSE(nn.Module):
  """ SimCSEのモデル """

  def __init__(
      self,
      base_model_name: str,
      mlp_only_train: bool = False,
      temperature: float = 0.05,
  ):
    """ モデルの初期化 """

    super().__init__()

    # モデル名からエンコーダを初期化
    self.encoder = AutoModel.from_pretrained(base_model_name)

    # MLP層の次元数
    self.hidden_size = self.encoder.config.hidden_size

    # MLP層の線形層
    self.dense = nn.Linear(self.hidden_size, self.hidden_size)

    # MLP層の活性化関数
    self.activation = nn.Tanh()

    # MLP層による変換を訓練時にのみ適用するよう設定するフラグ
    self.mlp_only_train = mlp_only_train

    # 交差エントロピー損失の計算時に使用する温度
    self.temperature = temperature

  def encode_texts(self, tokenized_texts: BatchEncoding) -> Tensor:
    """ エンコーダを用いて文をベクトルに変換 """

    # トークナイズされた分をエンコーダに入力
    encoded_texts = self.encoder(**tokenized_texts)

    # モデルの最終層の出力の[CLS]トークンのベクトルを取り出す
    encoded_texts = encoded_texts.last_hidden_state[:, 0]

    # フラグがTrueかつ学習時の時, MLP層の変換処理を適用せずにベクトルを返す
    if self.mlp_only_train and not self.training:
      return encoded_texts

    # MLP層の変換処理
    encoded_texts = self.dense(encoded_texts)
    encoded_texts = self.activation(encoded_texts)

    return encoded_texts

  def forward(
      self,
      tokenized_texts_1: BatchEncoding,
      tokenized_texts_2: BatchEncoding,
      labels: Tensor,
      label_scores: Tensor | None = None,
  ) -> ModelOutput:
    """ モデルの前向き計算を実施 """

    # 文のベクトルに変換
    encoded_texts_1 = self.encode_texts(tokenized_texts_1)
    encoded_texts_2 = self.encode_texts(tokenized_texts_2)

    # 文の類似度行列の計算
    sim_matrix = F.cosine_similarity(
        encoded_texts_1.unsqueeze(1),
        encoded_texts_2.unsqueeze(0),
        dim=2,
    )

    # 交差エントロピー損失を求める
    loss = F.cross_entropy(sim_matrix / self.temperature, labels)

    # 正例ペアに対するスコアを類似度行列から取り出す
    positive_mask = F.one_hot(labels, sim_matrix.size(1)).bool()
    positive_scores = torch.masked_select(sim_matrix, positive_mask)

    # 負例ペアに対するスコアを類似度行列から取り出す
    negative_mask = positive_mask
    negative_scores = torch.masked_select(sim_matrix, negative_mask)

    return ModelOutput(loss=loss, scores=positive_scores)

In [24]:
sup_model = SimCSE(base_model_name, mlp_only_train=False)

### Trainerの準備

In [25]:
from scipy.stats import spearmanr
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction) -> dict[str, float]:
  """
  モデルが予測したスコアと評価用データスコアの
  スピアマンの順位相関係数を計算
  """

  scores = p.predictions
  labels, label_scores = p.label_ids

  spearman = spearmanr(scores, label_scores).statistic

  return {"spearman": spearman}

In [26]:
from transformers import TrainingArguments, Trainer
sup_train_args = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    learning_rate=5e-5,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=250,
    logging_steps=250,
    save_steps=250,
    save_total_limit=1,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="spearman",
    remove_unused_columns=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
# Trainerを初期化
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import Trainer

class SimCSETrainer(Trainer):
  """ SimCSEのTrainer """

  def get_eval_dataloader(
      self, eval_dataset: Dataset | None = None
  )-> DataLoader:
    """
    検証・テストセットのDataLoaderでeval_collate_fnを使うように
    Trainerのget_eval_dataloaderをオーバーライド
    """

    if eval_dataset is None:
      eval_dataset = self.eval_dataset

    return DataLoader(
        eval_dataset,
        batch_size=64,
        collate_fn=eval_collate_fn,
        pin_memory=True,
    )

# 教師なしSimCSEのTrainerを初期化
sup_trainer = SimCSETrainer(
    model=sup_model,
    args=sup_train_args,
    data_collator=sup_train_collate_fn,
    train_dataset=sup_train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)


In [28]:
# 学習を実行
sup_trainer.train()

Step,Training Loss,Validation Loss,Spearman
250,0.995800,4.100932,0.640041
500,0.699000,3.552912,0.641851
750,0.839100,4.611025,0.598539
1000,0.960600,5.348977,0.559656
1250,1.064000,4.511310,0.619157
1500,0.972100,4.824152,0.626466
1750,0.992900,5.120217,0.628186
2000,0.740300,5.086644,0.635607
2250,0.740200,4.604506,0.656900
2500,0.898200,4.473429,0.637794


TrainOutput(global_step=4065, training_loss=0.7774550640861633, metrics={'train_runtime': 977.5849, 'train_samples_per_second': 532.244, 'train_steps_per_second': 4.158, 'total_flos': 0.0, 'train_loss': 0.7774550640861633, 'epoch': 3.0})

### 性能評価

In [29]:
# 検証セットで教師ありSimCSEのモデルの評価を行う
sup_trainer.evaluate(valid_dataset)

{'eval_loss': 4.60450553894043,
 'eval_spearman': 0.6569001463539875,
 'eval_runtime': 14.5875,
 'eval_samples_per_second': 853.538,
 'eval_steps_per_second': 6.718,
 'epoch': 3.0}

In [30]:
# テストセットで教師ありSimCSEのモデルの評価を行う
sup_trainer.evaluate(test_dataset)

{'eval_loss': 4.640908241271973,
 'eval_spearman': 0.657605089703987,
 'eval_runtime': 1.8603,
 'eval_samples_per_second': 783.199,
 'eval_steps_per_second': 6.451,
 'epoch': 3.0}